In [1]:
# import dependencies
import pandas as pd

In [2]:
# Create 8 dataframes, one for each .csv file
df1 = pd.read_csv('2018_01_01 to 2018_02_28.csv')
df2 = pd.read_csv('2018_03_01 to 2018_09_30.csv')
df3 = pd.read_csv('2018_10_01 to 2019_05_31.csv')
df4 = pd.read_csv('2019_06_01 to 2019_12_31.csv')
df5 = pd.read_csv('2020_01_01 to 2020_07_31.csv')
df6 = pd.read_csv('2020_08_01 to 2021_02_28.csv')
df7 = pd.read_csv('2021_03_01 to 2021_08_31.csv')
df8 = pd.read_csv('2021_09_01 to 2022_04_20.csv')

# Create a list of our 8 dataframes
df_list = [df1, df2, df3, df4, df5, df6, df7, df8]

master_data = pd.concat(df_list, ignore_index=True)
# master_data.tail()
master_data.to_csv('CountyMasterFile.csv')

In [3]:
# Start here. Read in the master file and drop the first column, which is a kind of duplicate
# index column.
full_df = pd.read_csv('CountyMasterFile.csv', usecols=lambda x: x != 'Unnamed: 0')
len(full_df)

20335

In [4]:
# Now I need to clean the dataset from potential 'fat-finger' typos, which have the effect
# of massive discrepancies between the original list price and the sales.

# Create a column that calculates the 'delta' between the original list price ('OLP') and the
# Sold Price, with the latter being a percentage of the former.
full_df['delta'] = full_df['Sold Price']/full_df['OLP']*100

# show the smallest deltas to see where a natural cutoff would be to simply drop values. As shown
# by the below smallest 22 values, it appears I should drop the bottom 19 values, or all rows
# where 'delta' is less than 28%.
full_df['delta'].sort_values(ascending=True).head(22)

6626      0.091587
13645     0.097361
17655     0.100000
18273     0.101227
9193      0.110873
7877      1.018213
10861     8.548791
18899     9.090909
5919      9.548059
383       9.578947
875       9.979541
16833    10.000000
2636     10.003031
11941    10.092521
12641    10.128492
12767    10.137976
17382    10.246721
17883    10.367816
10017    10.433333
13811    28.643216
11160    31.446541
10940    38.491147
Name: delta, dtype: float64

In [5]:
# We'll start by dropping all rows with delta values below minus 500%. We'll create a new
# 'version' of the dataframe as we go along, naming it an intermediate1 dataframe, or int1_df,
# for short. At each cleaning step, we'll increment the int_df by one, just so we create a 
# new copy each time until we finally arrive at a final cleaned dataframe.
int1_df = full_df.loc[full_df['delta'] > 28, :]
len(int1_df)

20316

In [6]:
# By sorting in the opposite direction, I can see there's a big difference in delta values in the
# positive direction when I sort descending, i.e., largest first. Based on the 25 largest values,
# we'll drop all values with a delta value larger than 300%.
int1_df['delta'].sort_values(ascending=False).head(25)

18847    120000.000000
14632    100000.000000
7043      97674.418605
11805     94795.539033
1090      94032.258065
7861      12833.333333
19881     10507.120528
13897      1072.000000
15873      1003.861004
84          982.456140
7238        969.557289
7177        926.229508
8382        925.704210
12414       909.090909
3413        846.560847
19815       236.722307
16066       231.114177
15074       181.746032
6607        181.095890
479         170.000000
12791       169.090909
19413       167.583792
2924        166.666667
18276       162.337662
10323       160.740741
Name: delta, dtype: float64

In [7]:
# Finally, our new dataframe, 'clean_df', will only feature rows with a corresponding delta value
# less than 300%.
int2_df = int1_df.loc[int1_df['delta'] < 300, :]
len(int2_df)

20301

In [8]:
# Explore the dataset by using the describe() method. Immediately, I notice that the 'DOM' column
# has a couple problems, with a min value of -77. Since that column measures Days On Market, 
# this should only be a minimum of 0, with no negative values.
int2_df.describe()

,Sold Price,OLP,List Price,Price,DOM,Lot Size Acres,Ttl Fin Ar,Year Blt,Beds,Baths Full,Baths Half,delta
count,2.030100e+04,2.030100e+04,2.030100e+04,2.030100e+04,20301.000000,19005.000000,20301.000000,20297.000000,20301.000000,20301.000000,20301.000000,20301.000000
mean,3.551218e+05,3.612302e+05,3.574733e+05,3.551195e+05,33.595537,35.144834,2290.050244,2001.729763,3.746466,2.621546,0.564455,98.786454
std,1.864104e+05,2.101470e+05,2.032690e+05,1.864139e+05,54.149340,2018.428223,1638.954313,14.967177,0.983110,0.895345,0.556596,6.164692
min,3.000000e+03,2.500000e+03,2.500000e+03,3.000000e+03,-77.000000,0.000000,0.000000,1800.000000,1.000000,0.000000,0.000000,28.643216
25%,2.500000e+05,2.519000e+05,2.500000e+05,2.500000e+05,4.000000,0.144000,1501.000000,1994.000000,3.000000,2.000000,0.000000,96.463023
50%,3.180000e+05,3.216100e+05,3.199000e+05,3.180000e+05,11.000000,0.331000,2147.000000,2004.000000,4.000000,2.000000,1.000000,99.587544
75%,4.120000e+05,4.194440e+05,4.149000e+05,4.120000e+05,41.000000,0.625000,2964.000000,2014.000000,4.000000,3.000000,1.000000,101.149425
max,9.250000e+06,1.350000e+07,1.350000e+07,9.250000e+06,1044.000000,250125.000000,59246.000000,2022.000000,9.000000,14.000000,10.000000,236.722307


In [9]:
# Instead of dropping any values with negative DOM values, I'll create a new column called 
# 'DOM_actual' where I'll subtract 'OMD' (off market date) from 'OnMtDate' to get the real number
# of days inbetween the date when it was taken off the market from when it was first listed.

# But first, cast these 2 columns as datetime64 variables so we can perform the operation.
int3_df = int2_df.copy()
int3_df["OnMktDate"] =  int2_df["OnMktDate"].astype('datetime64[ns]')
int3_df["OMD"] =  int2_df["OMD"].astype('datetime64[ns]')

int3_df['DOM_actual'] = (int3_df["OMD"] - int3_df["OnMktDate"]).dt.days
int3_df.describe()

,Sold Price,OLP,List Price,Price,DOM,Lot Size Acres,Ttl Fin Ar,Year Blt,Beds,Baths Full,Baths Half,delta,DOM_actual
count,2.030100e+04,2.030100e+04,2.030100e+04,2.030100e+04,20301.000000,19005.000000,20301.000000,20297.000000,20301.000000,20301.000000,20301.000000,20301.000000,20301.000000
mean,3.551218e+05,3.612302e+05,3.574733e+05,3.551195e+05,33.595537,35.144834,2290.050244,2001.729763,3.746466,2.621546,0.564455,98.786454,35.915374
std,1.864104e+05,2.101470e+05,2.032690e+05,1.864139e+05,54.149340,2018.428223,1638.954313,14.967177,0.983110,0.895345,0.556596,6.164692,56.324708
min,3.000000e+03,2.500000e+03,2.500000e+03,3.000000e+03,-77.000000,0.000000,0.000000,1800.000000,1.000000,0.000000,0.000000,28.643216,-77.000000
25%,2.500000e+05,2.519000e+05,2.500000e+05,2.500000e+05,4.000000,0.144000,1501.000000,1994.000000,3.000000,2.000000,0.000000,96.463023,4.000000
50%,3.180000e+05,3.216100e+05,3.199000e+05,3.180000e+05,11.000000,0.331000,2147.000000,2004.000000,4.000000,2.000000,1.000000,99.587544,13.000000
75%,4.120000e+05,4.194440e+05,4.149000e+05,4.120000e+05,41.000000,0.625000,2964.000000,2014.000000,4.000000,3.000000,1.000000,101.149425,45.000000
max,9.250000e+06,1.350000e+07,1.350000e+07,9.250000e+06,1044.000000,250125.000000,59246.000000,2022.000000,9.000000,14.000000,10.000000,236.722307,1044.000000


In [10]:
# There are still negative values for our 'DOM_actual' column, so we'll just drop these rows
# completely from the dataset by creating a new, slimmer dataframe ('int4_df')
int4_df = int3_df.copy()
int4_df = int4_df.loc[int4_df['DOM_actual'] >= 0]
len(int4_df)

20291

In [16]:
# Let's explore the dataset by grouping by high school, middle school, and elem school to first
# see how the average "DOM", or "days on the market," compares between the different schools

# When I group by the 'High Schl' column, I immediately see that homes zoned for River Ridge 
# and Etowah spend the least amount of time on the market.

int4_df.groupby('High Schl')[['delta','DOM_actual']].mean()

,delta,DOM_actual
High Schl,,
Cherokee,98.740655,38.619804
Creekview,97.579574,45.326957
Etowah,99.022053,29.431114
Other,97.610583,52.613208
River Ridge,99.642642,27.943566
Sequoyah,98.514173,39.922369
Woodstock,99.063331,34.497776


In [15]:
# Take a peek at the middle schools.
int4_df.groupby('Middle Schl')[['delta','DOM_actual']].mean()

,delta,DOM_actual
Middle Schl,,
Booth,99.004629,29.353576
Centre,97.872340,51.000000
Creekland,97.583415,45.350982
Dean Rusk,98.504159,39.822409
Freedom,98.938338,31.182286
Mill Creek,99.653697,28.025833
Other,97.673186,44.734694
Teasley,98.577371,43.237140
Woodstock,99.277916,32.706824


In [14]:
# Now for the elementary schools
int4_df.groupby('Elem Schl')[['delta','DOM_actual']].mean()

,delta,DOM_actual
Elem Schl,,
Arnold Mill,99.264102,32.436036
Avery,97.528081,45.314932
Ball Ground,97.837291,35.358000
Bascomb,98.373502,29.861797
Boston,99.147629,29.064368
Canton,96.983396,48.333333
Carmel,99.640108,26.714848
Centre,97.872340,51.000000
Clark Creek,99.428617,32.493857
